# Match halo id from catelog.h5 to its corresponding pRedshiftOutput????_fesc_halos.h5 file


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import h5py as h5
import yt
import pickle
import os
import math

In [1]:
import ytree

ImportError: cannot import name 'validate_index_order' from 'yt.data_objects.static_output' (/storage/home/hcoda1/9/bsherwin6/.local/lib/python3.9/site-packages/yt/data_objects/static_output.py)

In [14]:
#**********************************************************************
# Give ytree.load the full (absolute) path to johns tree_0_0_0.dat file
trees = ytree.load('rockstar_halos/trees/tree_0_0_0.dat')
#**********************************************************************
trees.add_alias_field('mass', 'Mvir')

NameError: name 'ytree' is not defined

In [ ]:
mrange = [1e8, 1e10]
halo_mmp = []
for tree in trees:
    if tree['mass'].in_units('Msun') < mrange[0] or tree['mass'].in_units('Msun') > mrange[1]:
        continue
    mmp = dict(center=[], mass=[], redshift=[], rvir=[])
    for prog in tree.pwalk():
        pos = yt.YTArray([prog['x'], prog['y'], prog['z']]) / trees.box_size
        mmp['center'].append(pos)
        mmp['mass'].append(prog['mass'].in_units('Msun'))
        mmp['redshift'].append(prog['redshift'])
        mmp['rvir'].append(prog['virial_radius'] / trees.box_size)
    mmp['center'] = yt.YTArray(mmp['center'])
    mmp['mass'] = yt.YTArray(mmp['mass'])
    mmp['redshift'] = yt.YTArray(mmp['redshift'])
    mmp['rvir'] = yt.YTArray(mmp['rvir'])
    halo_mmp.append(mmp)

In [ ]:
pfn = []
redshifts = []

#**********************************************************
# Give the full (absolute) path to johns redshifts.dat file
lines = open('redshifts.dat').readlines()
#**********************************************************
for l in lines:
    pfn.append(l.split(':')[0])
    redshifts.append(float(l.split('=')[-1]))

redshifts = np.array(redshifts)
pfn = np.array(pfn)
isort = redshifts.argsort().astype('int')[::-1]
redshifts = redshifts[isort]
pfn = pfn[isort]

In [ ]:

catalog = []
for fn in pfn:
    entry = {}
    #************************************************
    # Give the full path to johns fesc-data directory 
    fp = h5.File('fesc-data/p%s_fesc_halos.h5' % (fn.split('/')[1]), 'r')
    #************************************************    
    nhalos = len(fp.keys())
    if nhalos == 0:
        catalog.append(entry)
        fp.close()
        continue
    

    #gp0 = fp.values()[0]
    # avoid = ['Pop2_Stars', 'Pop3_Stars', 'fesc_bins', 'fesc_hist']
    # vector_fields = ['center']
    
    # # Create empty entry
    # for f in gp0:
    #     if f not in avoid:
    #         if f in vector_fields:
    #             entry[f] = np.zeros((nhalos,3))
    #         else:
    #             entry[f] = np.zeros(nhalos)    

    # Create empty entry (only need center and rvir)
    entry['center'] = np.zeros((nhalos,3))
    entry['rvir'] = np.zeros(nhalos)
                
    #----------------------------------------------
    entry['filename'] = np.empty(nhalos, dtype=str)
    #----------------------------------------------
    
    # # Read the data
    # i = 0
    # for g in fp.itervalues():
    #     for f in g:
    #         if f not in avoid:
    #             entry[f][i] = g[f].value
    #     i += 1    

    # Read the data
    i = 0
    for g in fp.itervalues():
        for f in g:
            if f in ['center', 'rvir']:
                entry[f][i] = g[f].value
        #---------------------------------          
        entry['filename'][i] = fp.filename 
        #---------------------------------          
        i += 1
                     
    catalog.append(entry)
    fp.close()


In [ ]:
def find_nearest(array, value):
    return np.abs(array - value).argmin()

In [ ]:
# Don't need this cuz we're not using other fields
# fields = catalog[-1].keys()
# fields.remove('center')
# fields.remove('rvir')

for halo in halo_mmp:
    nnode = len(halo['redshift'])
    # Don't need to add other fields
    # for f in fields:
    #     halo[f] = np.zeros(nnode)
    #---------------------------------          
    halo['filename'] = np.empty(nnode, dtype=str)
    #---------------------------------          
    
    for inode in range(nnode):
        ip = find_nearest(redshifts, halo['redshift'][inode])
        
        # dr = separation, R = radius of target halo, r = radii of all catalog halos
        dr = np.sqrt(((halo['center'][inode] - catalog[ip]['center'])**2).sum(1))
        R = halo['rvir'][inode]
        r = catalog[ip]['rvir'] * (1 + redshifts[ip]) / trees.box_size.in_units('kpc').v
        
        # Calculate the sphere-sphere intersection. The maximum is the match.
        # http://mathworld.wolfram.com/Sphere-SphereIntersection.html
        vol = np.pi * (R+r-dr)**2 * (dr**2 + 2*dr*r - 3*r**2 + 2*dr*R + 6*r*R - 3*R**2) / (12*dr)
        vol[dr > r+R] = 0.0  # No intersection
        vol[dr < R-r] = (4*np.pi/3) * r[dr < R-r]**3  # Complete intersection
        
        match = vol.argmax()
        rvir = catalog[ip]['rvir'][match] * (1 + redshifts[ip]) / trees.box_size.in_units('kpc').v
        # Check whether separation is within the virial radius
        if dr[match] >= rvir:
            continue
        # Don't need the fields, just the filename
        # # Add halo properties to mmp history
        # for f in fields:
        #     halo[f][inode] = catalog[ip][f][match]
        #---------------------------------          
        halo['filename'][inode] = catelog[ip]['filename'][match]
        #---------------------------------          
        
        
    # # Calculate SFR (last 20 Myr) [Msun/yr]
    # halo['SFR'] = halo['Mstar_pop2_young'] / 20e6
    # halo['fesc'] = np.maximum(halo['fesc'], 1e-6)

In [ ]:
fp = h5.File('catalog_filenames.h5', 'w')
for i,halo in enumerate(halo_mmp):
    hname = "Halo%8.8d" % (i)
    gp = fp.create_group(hname)
    for k,v in halo.items():
        gp[k] = v
fp.close()